# Trabalho 1 - Nivelamento

Considere os datasets abaixo, estratégias de pré-processamento, medidas de avaliação, métodos de comparação estatística e os seguintes algoritmos de aprendizado de máquina: árvore de decisão, random forest e k-nearest neighbor. A partir disso, responda as seguintes perguntas:

1. Qual o algoritmo de AM mais adequado para cada problema?
2. Qual o algoritmo de AM mais adequado para todos os problemas?
knn:85.94066985255232
Arvore de decisão:85.42638695604664
Random Florest:86.62282691074151
Partindo de uma comparação direta entre os resultados do cross validation usando o msm conjunto de dados para todos, o algoritmo de random florest é o melhor, no entanto é o mais demorado também demorando 31 minutos para processar tudo, enquanto so demais demoraram apenas 1 minuto cada, em segundo fica o knn e terceiro a arvore de decisão que tem tempos similares. As únicas alterações feitas no dataset foi a seleção de atributos, e a conversão de atributos nominais para numéricos/binários utilizando o getdummies ou a conversão direta para os que tinham apenas duas classes.

Para responder essas questões construa um notebook no colab ou um ambiente similar. Documente de forma clara cada passo e justifique suas decisões.

In [47]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
import matplotlib.patches as mpatches
import matplotlib as plt
from scipy.io import arff
import urllib.request
import pandas as pd
import numpy as np
import io

In [2]:
'''Link com os datasets'''
PhishingWebsites_link = "https://www.openml.org/data/download/1798106/phpV5QYya"
arrhythmia_link       = "https://www.openml.org/data/download/53551/arrhythmia.arff"
Satellite_link        = "https://www.openml.org/data/download/16787463/phpZrCzJR"
airlines_link         = "https://www.openml.org/data/download/66526/phpvcoG8S"
AedesSex_link         = "https://github.com/denismr/Classification-and-Counting-with-Recurrent-Contexts/raw/master/codeAndData/data/AedesSex.csv"
phoneme_link          = "https://www.openml.org/data/download/1592281/php8Mz7BG"
adult_link            = "https://www.openml.org/data/download/1595261/phpMawTba"

In [3]:
def loadDataset_Arff(url: str) -> pd.DataFrame:
  '''Função responsável por ler os datasets do tipo .arff'''
  ftpstream = urllib.request.urlopen(url)
  return pd.DataFrame(arff.loadarff(io.StringIO(ftpstream.read().decode('utf-8')))[0])
  
def get_dummies(df: pd.DataFrame, col:str) -> pd.DataFrame:
  '''Função responsável por separar os dados nominais em outras colunas.'''
  return pd.get_dummies(df,prefix=col,prefix_sep='.',columns=[col]).copy()

In [4]:
def proportion(Dataset,target="Target"):
  '''Função que gera um gráfico de proporção dos valores targets'''
  target_count = Dataset[target].value_counts()
  print('Class 0:', target_count[0])
  print('Class 1:', target_count[1])
  target_count.plot(kind='bar', title='Count (target)');
  
def normalize(dataset,target='Target') -> pd.DataFrame:
  '''Função que retorna o dataset normalizado'''
  X = dataset.drop([target], axis=1).copy()
  Y = dataset[target].copy()

  scaler = MinMaxScaler()
  X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
  dataset = X
  dataset[target] = Y
  return dataset

In [5]:
def knn(dataset,n_neighbors=3,target='Target'):
  '''Função que aplica o knn'''
  X_train, X_test, y_train, y_test = train_test_split(dataset.drop([target], axis=1), dataset[target], test_size=0.3)
  model = KNeighborsClassifier(n_neighbors=3)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print(accuracy_score(y_test, y_pred)*100)

def treeDecision(dataset,target='Target'):
  '''Função que aplica a arvore de decisão'''
  X_train, X_test, y_train, y_test = train_test_split(dataset.drop([target], axis=1), dataset[target], test_size=0.3)
  model = DecisionTreeClassifier()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print(accuracy_score(y_test, y_pred)*100)

def RForestDecision(dataset,target='Target'):
  '''Função que aplica random florest'''
  X_train, X_test, y_train, y_test = train_test_split(dataset.drop([target], axis=1), dataset[target], test_size=0.3)
  model = RandomForestClassifier()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print(accuracy_score(y_test, y_pred)*100)
  

In [6]:
def TTS_knn(dataset, x ,y ,n_neighbors=3):
  '''Função que aplica o knn com x e y previamente setados'''
  model = KNeighborsClassifier(n_neighbors=3)
  return cross_val_score(model, x, y, cv=20, scoring="accuracy")

def TTS_treeDecision(dataset, x ,y):
  '''Função que aplica a arvore de decisão com x e y previamente setados'''
  model = DecisionTreeClassifier()
  return cross_val_score(model, x, y, cv=20, scoring="accuracy")

def TTS_RForestDecision(dataset, x ,y):
  '''Função que aplica o random florest com x e y previamente setados'''
  model = RandomForestClassifier()
  return cross_val_score(model, x, y, cv=20, scoring="accuracy")

In [9]:
def selectAttributes(dataset):
  '''Função que aplica o knn com x e y previamente setados'''
  model = DecisionTreeClassifier()#max_leaf_nodes=10
  feature_ = RFE(model,n_features_to_select=5, step=1)
  return feature_.fit(dataset.drop(['Target'],axis=1),dataset['Target']).support_

def removeColumns(dataset):
  '''Função que faz a seleção de atributos e retorna uma lista com o nome das colunas que serão removidas'''
  attributes = selectAttributes(dataset)
  dropListIndex = np.where(attributes == False)[0]
  dropList = []
  for index in dropListIndex:
    if(dataset.columns[index] != "Target"):
      dropList.append(dataset.columns[index])
  return dropList


In [7]:
def underSampling(dataset):
  # CONTAR CLASSES
  count_class_1, count_class_0 = dataset.Target.value_counts()
  # # Divide by class
  df_class_0 = dataset[dataset['Target'] == 0]
  df_class_1 = dataset[dataset['Target'] == 1]
  df_class_1_under = df_class_1.sample(int(count_class_1*0.8))
  df_test_under = pd.concat([df_class_1_under, df_class_0], axis=0)
  return df_test_under

def overSampling(dataset):
  # CONTAR CLASSES
  count_class_1, count_class_0 = dataset.Target.value_counts()
  print(count_class_1, count_class_0)
  # # Divide by class
  df_class_0 = dataset[dataset['Target'] == 0]
  df_class_1 = dataset[dataset['Target'] == 1]
  df_class_0_over = df_class_0.sample(int(count_class_0/0.85),replace=True)
  df_test_over = pd.concat([df_class_0_over, df_class_1], axis=0)
  return df_test_over

# Lendo todos os datasets

In [8]:
InitDataset_PhishingWebsites = loadDataset_Arff(PhishingWebsites_link)
InitDataset_arrhythmia = loadDataset_Arff(arrhythmia_link)
InitDataset_Satellite = loadDataset_Arff(Satellite_link)
InitDataset_airlines = loadDataset_Arff(airlines_link)
InitDataset_AedesSex = pd.read_csv(AedesSex_link)
InitDataset_phoneme = loadDataset_Arff(phoneme_link)
InitDataset_adult = loadDataset_Arff(adult_link)

Abaixo será feito a conversão das classes targets para True e False, e trocado o nome da coluna para Target para as que não estão. Também será feito uma analise de distribuição e procura de dados nominais.

In [10]:
Dataset_PhishingWebsites = InitDataset_PhishingWebsites.copy()
Dataset_PhishingWebsites['Target'] = [True if x == '-1'.encode() else False for x in Dataset_PhishingWebsites['Result']]
Dataset_PhishingWebsites = Dataset_PhishingWebsites.drop(['Result'],axis=1)
Dataset_PhishingWebsites = Dataset_PhishingWebsites.drop(removeColumns(Dataset_PhishingWebsites),axis=1)

In [14]:
Dataset_arrhythmia = InitDataset_arrhythmia.copy()
Dataset_arrhythmia['Target'] = [True if x == 'P'.encode() else False for x in Dataset_arrhythmia['binaryClass']]
Dataset_arrhythmia = Dataset_arrhythmia.drop(['binaryClass'],axis=1) #Remoção da coluna J por ter 98% dos dados NaN
Dataset_arrhythmia = Dataset_arrhythmia.dropna() #Remoção das linhas com valores faltantes
Dataset_arrhythmia = Dataset_arrhythmia.drop(removeColumns(Dataset_arrhythmia),axis=1) 
Dataset_arrhythmia.reset_index(inplace=True, drop=True)

In [15]:
##Sem nominais usando a função select_nominals
Dataset_Satellite = InitDataset_Satellite.copy()
Dataset_Satellite['Target'] = [True if x == 'Anomaly'.encode() else False for x in Dataset_Satellite['Target']]
Dataset_Satellite = Dataset_Satellite.drop(removeColumns(Dataset_Satellite),axis=1) 

In [16]:
##Dataset com muitos dados nominais, para aplicar o knn será necessário usar a função getdummies em AirportFrom e AirportTo.
Dataset_airlines = InitDataset_airlines.copy()
Dataset_airlines['Target'] = [True if x == '1'.encode() else False for x in Dataset_airlines['Delay']]
Dataset_airlines = Dataset_airlines.drop(['Delay'],axis=1)

Dataset_airlines = Dataset_airlines.drop_duplicates()


Dataset_airlines['Airline'] = Dataset_airlines['Airline'].astype('category')
Dataset_airlines['AirportFrom'] = Dataset_airlines['AirportFrom'].astype('category')
Dataset_airlines['AirportTo'] = Dataset_airlines['AirportTo'].astype('category')

Dataset_airlines['Airline'] = Dataset_airlines['Airline'].cat.codes
Dataset_airlines['AirportFrom'] = Dataset_airlines['AirportFrom'].cat.codes
Dataset_airlines['AirportTo'] = Dataset_airlines['AirportTo'].cat.codes

Dataset_airlines = Dataset_airlines.drop(removeColumns(Dataset_airlines),axis=1)
Dataset_airlines.reset_index(inplace=True, drop=True)

In [17]:
Dataset_AedesSex = InitDataset_AedesSex.copy()
Dataset_AedesSex['Target'] = [True if x == 'F' else False for x in Dataset_AedesSex['sex']]
Dataset_AedesSex = Dataset_AedesSex.drop(['sex'],axis=1)
Dataset_AedesSex = Dataset_AedesSex.drop(removeColumns(Dataset_AedesSex),axis=1) 

In [28]:
Dataset_phoneme = InitDataset_phoneme.copy()
Dataset_phoneme['Target'] = [True if x == '1'.encode() else False for x in Dataset_phoneme['Class']]
Dataset_phoneme = Dataset_phoneme.drop(['Class'],axis=1)
Dataset_phoneme = Dataset_phoneme.drop(removeColumns(Dataset_phoneme),axis=1)
Dataset_phoneme = overSampling(underSampling(Dataset_phoneme))#Aplicando over e undersampling para 15% dos dados para tentar deixar mais proporcional o dataset
Dataset_phoneme.reset_index(inplace=True, drop=True)

3054 1586


In [24]:
Dataset_adult = InitDataset_adult.copy()
Dataset_adult['Target'] = [True if x == '<=50K'.encode() else False for x in Dataset_adult['class']]
Dataset_adult = Dataset_adult.drop(['class'],axis=1)
Dataset_adult['workclass'] = [x.decode('UTF-8') for x in Dataset_adult['workclass']]
Dataset_adult['education'] = [x.decode('UTF-8') for x in Dataset_adult['education']]
Dataset_adult['marital-status'] = [x.decode('UTF-8') for x in Dataset_adult['marital-status']]
Dataset_adult['occupation'] = [x.decode('UTF-8') for x in Dataset_adult['occupation']]
Dataset_adult['relationship'] = [x.decode('UTF-8') for x in Dataset_adult['relationship']]
Dataset_adult['race'] = [x.decode('UTF-8') for x in Dataset_adult['race']]
Dataset_adult['sex'] = [x.decode('UTF-8') for x in Dataset_adult['sex']]
Dataset_adult['native-country'] = [x.decode('UTF-8') for x in Dataset_adult['native-country']]

Dataset_adult = Dataset_adult.dropna() #Remoção das linhas com valores faltantes
Dataset_adult = get_dummies(Dataset_adult,'workclass')
Dataset_adult = get_dummies(Dataset_adult,'education')
Dataset_adult = get_dummies(Dataset_adult,'marital-status')
Dataset_adult = get_dummies(Dataset_adult,'occupation')
Dataset_adult = get_dummies(Dataset_adult,'relationship')
Dataset_adult = get_dummies(Dataset_adult,'race')
Dataset_adult = get_dummies(Dataset_adult,'sex')
Dataset_adult = get_dummies(Dataset_adult,'native-country')
Dataset_adult = Dataset_adult.drop(removeColumns(Dataset_adult),axis=1)

In [29]:
##Normalizando todos os dados
NOR_Dataset_PhishingWebsites = normalize(Dataset_PhishingWebsites)
NOR_Dataset_arrhythmia = normalize(Dataset_arrhythmia)
NOR_Dataset_Satellite = normalize(Dataset_Satellite)
NOR_Dataset_AedesSex = normalize(Dataset_AedesSex)
NOR_Dataset_airlines = normalize(Dataset_airlines)
NOR_Dataset_phoneme = normalize(Dataset_phoneme)
NOR_Dataset_adult = normalize(Dataset_adult)

In [ ]:
# knn(NOR_Dataset_PhishingWebsites) #93.8498643352427
knn(NOR_Dataset_arrhythmia) 
# knn(NOR_Dataset_Satellite) #99.2156862745098
# knn(NOR_Dataset_AedesSex) #98.73611111111111
# knn(NOR_Dataset_phoneme) #88.03945745992601
# knn(NOR_Dataset_adult) #81.71022998703337
# knn(NOR_Dataset_airlines) #81.71022998703337


# treeDecision(NOR_Dataset_PhishingWebsites)#96.02050045221586
# treeDecision(NOR_Dataset_arrhythmia)
# treeDecision(NOR_Dataset_Satellite)#99.08496732026144
# treeDecision(NOR_Dataset_AedesSex)#98.19444444444444
# treeDecision(NOR_Dataset_phoneme)#87.05302096177559
# treeDecision(NOR_Dataset_adult)#82.01733433426602
# treeDecision(NOR_Dataset_airlines)


# RForestDecision(NOR_Dataset_PhishingWebsites)#96.80434127223396
# RForestDecision(NOR_Dataset_arrhythmia)
# RForestDecision(NOR_Dataset_Satellite)#99.281045751634
# RForestDecision(NOR_Dataset_AedesSex)#98.38888888888889
# RForestDecision(NOR_Dataset_phoneme)#90.9987669543773
# RForestDecision(NOR_Dataset_adult)#85.68893741895857
# RForestDecision(NOR_Dataset_airlines)



In [31]:
##Separando os atributos dos datasets para testar qual o melhor algoritmo
NOR_Dataset_PhishingWebsites_y = NOR_Dataset_PhishingWebsites['Target']
NOR_Dataset_PhishingWebsites_x = NOR_Dataset_PhishingWebsites.drop(['Target'],axis=1)

NOR_Dataset_Satellite_y = NOR_Dataset_Satellite['Target']
NOR_Dataset_Satellite_x = NOR_Dataset_Satellite.drop(['Target'],axis=1)

NOR_Dataset_AedesSex_y = NOR_Dataset_AedesSex['Target']
NOR_Dataset_AedesSex_x = NOR_Dataset_AedesSex.drop(['Target'],axis=1)

NOR_Dataset_phoneme_y = NOR_Dataset_phoneme['Target']
NOR_Dataset_phoneme_x = NOR_Dataset_phoneme.drop(['Target'],axis=1)

NOR_Dataset_adult_y = NOR_Dataset_adult['Target']
NOR_Dataset_adult_x = NOR_Dataset_adult.drop(['Target'],axis=1)

NOR_Dataset_airlines_y = NOR_Dataset_airlines['Target']
NOR_Dataset_airlines_x = NOR_Dataset_airlines.drop(['Target'],axis=1)

NOR_Dataset_arrhythmia_y = NOR_Dataset_arrhythmia['Target']
NOR_Dataset_arrhythmia_x = NOR_Dataset_arrhythmia.drop(['Target'],axis=1)

In [35]:
##Calculando a media de todos os datasets a partir do cross validation juntamente com o knn
resultCrossValidationKnnDataset_PhishingWebsites = TTS_knn(NOR_Dataset_PhishingWebsites,NOR_Dataset_PhishingWebsites_x,NOR_Dataset_PhishingWebsites_y) 
resultCrossValidationKnnDataset_Satellite = TTS_knn(NOR_Dataset_Satellite,NOR_Dataset_Satellite_x,NOR_Dataset_Satellite_y) 
resultCrossValidationKnnDataset_AedesSex = TTS_knn(NOR_Dataset_AedesSex,NOR_Dataset_AedesSex_x,NOR_Dataset_AedesSex_y) 
resultCrossValidationKnnDataset_phoneme = TTS_knn(NOR_Dataset_phoneme,NOR_Dataset_phoneme_x,NOR_Dataset_phoneme_y) 
resultCrossValidationKnnDataset_adult = TTS_knn(NOR_Dataset_adult,NOR_Dataset_adult_x,NOR_Dataset_adult_y) 
resultCrossValidationKnnDataset_airlines = TTS_knn(NOR_Dataset_airlines,NOR_Dataset_airlines_x,NOR_Dataset_airlines_y) 
resultCrossValidationKnnDataset_arrhythmia = TTS_knn(NOR_Dataset_arrhythmia,NOR_Dataset_arrhythmia_x,NOR_Dataset_arrhythmia_y) 

In [46]:
print(resultCrossValidationKnnDataset_PhishingWebsites,
resultCrossValidationKnnDataset_Satellite,
resultCrossValidationKnnDataset_AedesSex,
resultCrossValidationKnnDataset_phoneme,
resultCrossValidationKnnDataset_adult,
resultCrossValidationKnnDataset_airlines,
resultCrossValidationKnnDataset_arrhythmia)

[0.90596745 0.92766727 0.93490054 0.92585895 0.93851718 0.92947559
 0.94575045 0.91139241 0.92766727 0.92947559 0.9403255  0.95298373
 0.91681736 0.93670886 0.93128391 0.86050725 0.89492754 0.88768116
 0.90036232 0.89492754] [0.99607843 0.99607843 0.99215686 0.98823529 1.         0.99607843
 0.99607843 0.99607843 0.99607843 0.99215686 0.99607843 0.99607843
 0.98823529 0.98823529 0.99607843 0.99607843 0.99607843 0.98823529
 0.99215686 0.99215686] [0.9725     0.93833333 0.98       0.99       0.98083333 0.9825
 0.98583333 0.9925     0.99166667 0.9825     0.94666667 0.98083333
 0.98833333 0.98833333 0.99       0.9825     0.98666667 0.9875
 0.99       0.99416667] [0.90243902 0.89430894 0.90243902 0.91869919 0.90650407 0.87398374
 0.87804878 0.86585366 0.91869919 0.89837398 0.90243902 0.89430894
 0.90243902 0.89837398 0.88211382 0.91056911 0.87804878 0.91056911
 0.84552846 0.88163265] [0.78141629 0.76217765 0.77846028 0.77395577 0.77436527 0.77108927
 0.77600328 0.75962326 0.78869779 0.79197

In [36]:
##Calculando a media de todos os datasets a partir do cross validation juntamente com a árvore de decisão
resultCrossValidationTreeDecisionDataset_PhishingWebsites = TTS_treeDecision(NOR_Dataset_PhishingWebsites,NOR_Dataset_PhishingWebsites_x,NOR_Dataset_PhishingWebsites_y) 
resultCrossValidationTreeDecisionDataset_Satellite = TTS_treeDecision(NOR_Dataset_Satellite,NOR_Dataset_Satellite_x,NOR_Dataset_Satellite_y) 
resultCrossValidationTreeDecisionDataset_AedesSex = TTS_treeDecision(NOR_Dataset_AedesSex,NOR_Dataset_AedesSex_x,NOR_Dataset_AedesSex_y) 
resultCrossValidationTreeDecisionDataset_phoneme = TTS_treeDecision(NOR_Dataset_phoneme,NOR_Dataset_phoneme_x,NOR_Dataset_phoneme_y) 
resultCrossValidationTreeDecisionDataset_adult = TTS_treeDecision(NOR_Dataset_adult,NOR_Dataset_adult_x,NOR_Dataset_adult_y) 
resultCrossValidationTreeDecisionDataset_airlines = TTS_treeDecision(NOR_Dataset_airlines,NOR_Dataset_airlines_x,NOR_Dataset_airlines_y) 
resultCrossValidationTreeDecisionDataset_arrhythmia = TTS_treeDecision(NOR_Dataset_arrhythmia,NOR_Dataset_arrhythmia_x,NOR_Dataset_arrhythmia_y) 

In [45]:
print(resultCrossValidationTreeDecisionDataset_PhishingWebsites,
resultCrossValidationTreeDecisionDataset_Satellite,
resultCrossValidationTreeDecisionDataset_AedesSex,
resultCrossValidationTreeDecisionDataset_phoneme,
resultCrossValidationTreeDecisionDataset_adult,
resultCrossValidationTreeDecisionDataset_airlines,
resultCrossValidationTreeDecisionDataset_arrhythmia)

[0.90958409 0.93128391 0.91862568 0.92585895 0.93670886 0.90958409
 0.93851718 0.91500904 0.90958409 0.93128391 0.93128391 0.94936709
 0.920434   0.93490054 0.92947559 0.87137681 0.93297101 0.91485507
 0.92934783 0.92934783] [0.99607843 0.99607843 0.99215686 0.98823529 0.98823529 0.98823529
 0.99607843 0.99215686 0.97647059 0.98431373 0.99607843 0.99215686
 0.99215686 0.99215686 0.99607843 0.99607843 0.99607843 0.98431373
 0.98431373 0.98431373] [0.955      0.92583333 0.96416667 0.9775     0.98333333 0.98416667
 0.9825     0.9925     0.9925     0.99416667 0.9275     0.97
 0.98166667 0.9825     0.99       0.98083333 0.98583333 0.98333333
 0.98916667 0.99416667] [0.89430894 0.8902439  0.8902439  0.91056911 0.92682927 0.90243902
 0.91056911 0.93902439 0.88617886 0.91056911 0.91869919 0.90243902
 0.90243902 0.91463415 0.89837398 0.90650407 0.91056911 0.92276423
 0.91056911 0.88571429] [0.75521899 0.76135898 0.75307125 0.76248976 0.74406224 0.75593776
 0.76453726 0.75921376 0.75307125 0.763

In [37]:
##Calculando a media de todos os datasets a partir do cross validation juntamente com o algoritmo de random florest.
resultCrossValidationRFDataset_PhishingWebsites = TTS_RForestDecision(NOR_Dataset_PhishingWebsites,NOR_Dataset_PhishingWebsites_x,NOR_Dataset_PhishingWebsites_y) 
resultCrossValidationRFDataset_Satellite = TTS_RForestDecision(NOR_Dataset_Satellite,NOR_Dataset_Satellite_x,NOR_Dataset_Satellite_y) 
resultCrossValidationRFDataset_AedesSex = TTS_RForestDecision(NOR_Dataset_AedesSex,NOR_Dataset_AedesSex_x,NOR_Dataset_AedesSex_y) 
resultCrossValidationRFDataset_phoneme = TTS_RForestDecision(NOR_Dataset_phoneme,NOR_Dataset_phoneme_x,NOR_Dataset_phoneme_y) 
resultCrossValidationRFDataset_adult = TTS_RForestDecision(NOR_Dataset_adult,NOR_Dataset_adult_x,NOR_Dataset_adult_y) 
resultCrossValidationRFDataset_airlines = TTS_RForestDecision(NOR_Dataset_airlines,NOR_Dataset_airlines_x,NOR_Dataset_airlines_y) 
resultCrossValidationRFDataset_arrhythmia = TTS_RForestDecision(NOR_Dataset_arrhythmia,NOR_Dataset_arrhythmia_x,NOR_Dataset_arrhythmia_y) 

In [43]:
print(resultCrossValidationRFDataset_PhishingWebsites,
resultCrossValidationRFDataset_Satellite,
resultCrossValidationRFDataset_AedesSex,
resultCrossValidationRFDataset_phoneme,
resultCrossValidationRFDataset_adult,
resultCrossValidationRFDataset_airlines,
resultCrossValidationRFDataset_arrhythmia)

[0.90958409 0.93128391 0.920434   0.92585895 0.93670886 0.90958409
 0.93851718 0.91500904 0.92585895 0.93128391 0.93128391 0.94936709
 0.920434   0.93490054 0.92947559 0.86594203 0.93297101 0.91666667
 0.92572464 0.92934783] [0.99607843 0.99607843 0.99215686 0.98823529 0.99607843 0.99607843
 0.99607843 0.99215686 0.99607843 0.98823529 1.         0.99215686
 0.98823529 0.99215686 0.99607843 0.99607843 0.99607843 0.99215686
 0.99607843 0.99215686] [0.97416667 0.94583333 0.97666667 0.98916667 0.99083333 0.99083333
 0.9875     0.995      0.99583333 0.99333333 0.955      0.98083333
 0.98666667 0.98916667 0.99083333 0.99083333 0.9875     0.98666667
 0.99333333 0.99666667] [0.94308943 0.92682927 0.93089431 0.95121951 0.95934959 0.91869919
 0.93495935 0.94308943 0.92276423 0.94308943 0.94308943 0.93902439
 0.92682927 0.93495935 0.90650407 0.93495935 0.93902439 0.93089431
 0.92682927 0.95102041] [0.77568563 0.76872698 0.77395577 0.77231777 0.77067977 0.78337428
 0.77764128 0.77682228 0.78746929

In [54]:
from scipy.stats import friedmanchisquare


if friedmanchisquare(resultCrossValidationTreeDecisionDataset_PhishingWebsites,resultCrossValidationRFDataset_PhishingWebsites,resultCrossValidationKnnDataset_PhishingWebsites)[1] > 0.05:
  print("Os algoritmos para PhishingWebsites são muito parecidos ou iguais.")

if friedmanchisquare(resultCrossValidationTreeDecisionDataset_Satellite,resultCrossValidationRFDataset_Satellite,resultCrossValidationKnnDataset_Satellite)[1] > 0.05:
  print("Os algoritmos para Satellite são muito parecidos ou iguais.")

if friedmanchisquare(resultCrossValidationTreeDecisionDataset_AedesSex,resultCrossValidationRFDataset_AedesSex,resultCrossValidationKnnDataset_AedesSex)[1] > 0.05:
  print("Os algoritmos para AedesSex são muito parecidos ou iguais.")

if friedmanchisquare(resultCrossValidationTreeDecisionDataset_phoneme,resultCrossValidationRFDataset_phoneme,resultCrossValidationKnnDataset_phoneme)[1] > 0.05:
  print("Os algoritmos para phoneme são muito parecidos ou iguais.")

if friedmanchisquare(resultCrossValidationTreeDecisionDataset_adult,resultCrossValidationRFDataset_adult,resultCrossValidationKnnDataset_adult)[1] > 0.05:
  print("Os algoritmos para adult são muito parecidos ou iguais.")

if friedmanchisquare(resultCrossValidationTreeDecisionDataset_airlines,resultCrossValidationRFDataset_airlines,resultCrossValidationKnnDataset_airlines)[1] > 0.05:
  print("Os algoritmos para airlines são muito parecidos ou iguais.")

if friedmanchisquare(resultCrossValidationTreeDecisionDataset_arrhythmia,resultCrossValidationRFDataset_arrhythmia,resultCrossValidationKnnDataset_arrhythmia)[1] > 0.05:
  print("Os algoritmos para arrhythmia são muito parecidos ou iguais.")


Os algoritmos para PhishingWebsites são muito parecidos ou iguais.
0.02534940552272465
